In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse import csr_matrix

In [2]:
file  = np.load('user_movie_rating.npy')
user_id = file[:,0]
movie_id = file[:,1]
rating = file[:,2]

similarity_threshold = 0.5
print(len(movie_id))

65225506


In [3]:
#create a sparse matrix of the user-movie rating, where the movies that recieved a rating get a 1 and the ones that did not get a rating get a 0
#We assume from the assignment that we only care about movies that were rated but not the actual ratings. Thus giving a binary matrix

def create_matrix(file):
    user_id = file[:,0]
    movie_id = file[:,1]
    rating = file[:,2]
    data = np.ones(len(rating))
    user_movie_matrix = csc_matrix((data, (user_id, movie_id)))
    return user_movie_matrix

In [4]:
user_movie_matrix = create_matrix(file)
#delete the first row and column since they are empty
user_movie_matrix = user_movie_matrix[1:,1:]
print(user_movie_matrix)
print(user_movie_matrix.shape)

  (94, 0)	1.0
  (154, 0)	1.0
  (440, 0)	1.0
  (574, 0)	1.0
  (852, 0)	1.0
  (1188, 0)	1.0
  (1214, 0)	1.0
  (1263, 0)	1.0
  (1498, 0)	1.0
  (1716, 0)	1.0
  (1717, 0)	1.0
  (1802, 0)	1.0
  (2180, 0)	1.0
  (2325, 0)	1.0
  (2680, 0)	1.0
  (3399, 0)	1.0
  (3669, 0)	1.0
  (3689, 0)	1.0
  (3880, 0)	1.0
  (4755, 0)	1.0
  (4798, 0)	1.0
  (5267, 0)	1.0
  (5690, 0)	1.0
  (5883, 0)	1.0
  (6741, 0)	1.0
  :	:
  (100303, 17769)	1.0
  (100390, 17769)	1.0
  (100506, 17769)	1.0
  (100559, 17769)	1.0
  (100609, 17769)	1.0
  (100663, 17769)	1.0
  (100665, 17769)	1.0
  (100700, 17769)	1.0
  (100719, 17769)	1.0
  (100730, 17769)	1.0
  (100763, 17769)	1.0
  (101075, 17769)	1.0
  (101297, 17769)	1.0
  (101858, 17769)	1.0
  (101922, 17769)	1.0
  (102016, 17769)	1.0
  (102036, 17769)	1.0
  (102106, 17769)	1.0
  (102255, 17769)	1.0
  (102579, 17769)	1.0
  (103054, 17769)	1.0
  (103107, 17769)	1.0
  (103343, 17769)	1.0
  (103381, 17769)	1.0
  (103441, 17769)	1.0
(103703, 17770)


In [5]:
#make a minhash function for the csc matrix, I realise now that a csr matrix might have been more efficient
def minhash(data, num_permutations):
    if not isinstance(data, csc_matrix):
        raise ValueError("Input data must be a scipy.sparse CSC matrix.")
    
    num_movies = data.shape[1]  # Number of columns (features)
    num_users = data.shape[0]   # Number of rows (samples)
    
    # Create random permutations
    permutations = [np.random.permutation(num_movies) for _ in range(num_permutations)]
    perm_signature = np.full((num_users, num_permutations), np.inf)
    
    # Iterate over columns (movies/features) of the CSC matrix
    for movie_index in range(num_movies):
        # Get all non-zero row indices for the current column
        row_indices = data.indices[data.indptr[movie_index]:data.indptr[movie_index + 1]]
        
        for k in range(num_permutations):
            # Compute the hash value for the current column (movie_index)
            hash_value = permutations[k][movie_index]
            
            # Update the MinHash signature for the rows with non-zero values in this column
            for row_index in row_indices:
                perm_signature[row_index, k] = min(perm_signature[row_index, k], hash_value)

    return perm_signature.transpose()



In [ ]:
# testhash = minhash(user_movie_matrix, 100)

# #save the minhash signature to a file
# np.save('minhash.npy', testhash)
testhash = np.load('minhash.npy')

In [7]:
print(testhash)

[[ 54.  40.  40. ...  40.  16.  40.]
 [ 32.   9.   9. ...  23.  13.  32.]
 [ 30.  92.  11. ... 514.  92. 200.]
 ...
 [  4.   3.   3. ...   3.   3.   3.]
 [  7.   7.  10. ... 139.  31. 227.]
 [  6.   6.  27. ...  48.  71.  62.]]


In [30]:
#print the first column from the user_movie_matrix, so for the first person
print(user_movie_matrix[0,:])
print(testhash[:,0])


[inf inf inf]
